In [1]:
#Check Python Version
import shutil
import sys
sys.version

'3.8.2 (default, Jan 31 2023, 18:34:03) \n[GCC 12.2.0]'

In [2]:
#Upgrade pip
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Check TF & TFX Versioning
import tensorflow as tf
print(tf.__version__)
from tfx import v1 as tfx
print(tfx.__version__)


2023-02-14 01:38:29.248990: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 01:38:29.429326: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-14 01:38:29.429394: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-14 01:38:30.766820: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

2.11.0
1.12.0


In [4]:
#Setup Variables as examplegen_playground
import os

# Pipeline name
PIPELINE_NAME = "importer_node_playground"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = './artifacts'
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# Folder path to data
DATA_ROOT = './data/'

# Global variable for ExampleGen component
EXAMPLE_GEN = tfx.components.CsvExampleGen


from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

In [5]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext()

# Importer Node
examples_importer = tfx.dsl.Importer(
    source_uri='../artifacts/CopyExampleGen/examples/2/1/',
    artifact_type=tfx.types.standard_artifacts.Examples).with_id(
        'examples_importer')

context.run(examples_importer)

INFO:absl:Running driver for examples_importer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: ../artifacts/CopyExampleGen/examples/2/1/, properties: {}, custom_properties: {}
INFO:absl:Running executor for examples_importer
INFO:absl:Running publisher for examples_importer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: examples_importer
    execution_id: 1
    outputs:
        result: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "../artifacts/CopyExampleGen/examples/2/1/"
        custom_properties {
          key: "is_external"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.12.0"
          }
        }
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [6]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext()

# Importer Node
# Brings data into the pipeline.
example_gen = tfx.components.CsvExampleGen(input_base='./data/')

context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data ./data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [7]:
###################################################################################
### [USE CASE 1]
### Tfrecords are already created but not in Examples Artifact form

### This component will:
### 1. Accept a dict with with {'split_name': './path/to/split_name/tfrecords.gz'}
### 2. Add them to a folder to follow Example Artifact directory structure
### 3. Import them with Importer node to register the external resource into MLMD
###################################################################################
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Create pipeline to run Importer node
def _create_pipeline(
  pipeline_name: str,
  pipeline_root: str,
  data_root: str,
  metadata_path: str
  ) -> tfx.dsl.Pipeline:

  context = InteractiveContext()
  ### Get external source data
  # Source directory of external Examples Artifacts
  source_examples_artifact_uri = '../artifacts/'

  # Destination directory for source
  destination_examples_artifact_uri = './artifacts/'

  # Get all files from source_examples_artifact_uri
  files = os.listdir(source_examples_artifact_uri)

  # Import source files to destination
  shutil.copytree(source_examples_artifact_uri, destination_examples_artifact_uri, dirs_exist_ok=True)


  # example_gen = tfx.components.CsvExampleGen(input_base=data_root)
  # print(example_gen.outputs['examples'])

  # Importer Node
  examples_importer = tfx.dsl.Importer(
      source_uri='../artifacts/CopyExampleGen/examples/2/1/',
      artifact_type=tfx.types.standard_artifacts.Examples).with_id(
          'examples_importer')

  context.run(examples_importer)
  print("IMPORTER_NODE: ", examples_importer.outputs)
  print("IMPORTER_NODE: ", examples_importer.outputs['result'].additional_properties.keys)
  print(list(examples_importer.outputs['result'].additional_properties.keys()))


  # Test if downstream component accepts output from Importer Node
  statistics_gen = tfx.components.StatisticsGen(
      examples=examples_importer.outputs['result'])


  # EXAMPLEGEN COMPONENT OUTPUT
  # Brings data into the pipeline.
  # example_gen = tfx.components.CsvExampleGen(input_base=data_root)
  # print("EXAMPLE_GEN: ", example_gen.outputs['examples'])

  # print("tfx.components.CsvExampleGen.outputs['examples'] output: \n\n",
  #   "Artifact Type: ", example_gen.outputs['examples']._artifact_type, "\n"
  #   "Producer Component Id: ", example_gen.outputs['examples'].producer_component_id, "\n"
  #   "Output Key: ", example_gen.outputs['examples'].output_key, "\n"
  #   "Additional Properties: ", example_gen.outputs['examples'].additional_properties, "\n"
  #   "Additional Custom Properties: ", example_gen.outputs['examples'].additional_custom_properties, "\n"
  # )
  # print(list(example_gen.outputs['examples'].additional_properties.keys()))

  components = [
    # example_gen,
    examples_importer,
    statistics_gen
  ]

  return tfx.dsl.Pipeline(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
    components=components
)

In [9]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=METADATA_PATH)
  )

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Using deployment config:
 executor_specs {
  key: "StatisticsGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.statistics_gen.executor.Executor"
      }
    }
  }
}
metadata_connection_config {
  database_connection_config {
    sqlite {
      filename_uri: "metadata/importer_node_playground/metadata.db"
      connection_mode: READWRITE_OPENCREATE
    }
  }
}

INFO:absl:Using connection config:
 sqlite {
  filename_uri: "metadata/importer_node_playground/metadata.db"
  connection_mode: READWRITE_OPENCREATE
}

INFO:absl:Component examples_importer is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.dsl.components.common.importer.Importer"
  }
  id: "examples_importer"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "importer_node_playground"
      }
    }
  }
 

IMPORTER_NODE:  {'result': OutputChannel(artifact_type=Examples, producer_component_id=examples_importer, output_key=result, additional_properties={}, additional_custom_properties={})}
IMPORTER_NODE:  <built-in method keys of dict object at 0x7f3af98eb100>
[]


ValueError: No splits for examples artifact: Artifact(artifact: id: 18
type_id: 14
uri: "../artifacts/CopyExampleGen/examples/2/1/"
custom_properties {
  key: "is_external"
  value {
    int_value: 1
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.12.0"
  }
}
state: LIVE
create_time_since_epoch: 1676047493360
last_update_time_since_epoch: 1676047493360
, artifact_type: id: 14
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
base_type: DATASET
)

In [8]:
import re

dict_output = {
	"name": "ValueError",
	"message": "No splits for examples artifact: Artifact(artifact: id: 18\ntype_id: 14\nuri: \"../artifacts/CopyExampleGen/examples/2/1/\"\ncustom_properties {\n  key: \"is_external\"\n  value {\n    int_value: 1\n  }\n}\ncustom_properties {\n  key: \"state\"\n  value {\n    string_value: \"published\"\n  }\n}\ncustom_properties {\n  key: \"tfx_version\"\n  value {\n    string_value: \"1.12.0\"\n  }\n}\nstate: LIVE\ncreate_time_since_epoch: 1676047493360\nlast_update_time_since_epoch: 1676047493360\n, artifact_type: id: 14\nname: \"Examples\"\nproperties {\n  key: \"span\"\n  value: INT\n}\nproperties {\n  key: \"split_names\"\n  value: STRING\n}\nproperties {\n  key: \"version\"\n  value: INT\n}\nbase_type: DATASET\n)",
	"stack": "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m\n\u001b[0;31mValueError\u001b[0m                                Traceback (most recent call last)\n\u001b[0;32m/tmp/ipykernel_2151988/3420384479.py\u001b[0m in \u001b[0;36m<cell line: 1>\u001b[0;34m()\u001b[0m\n\u001b[0;32m----> 1\u001b[0;31m tfx.orchestration.LocalDagRunner().run(\n\u001b[0m\u001b[1;32m      2\u001b[0m   _create_pipeline(\n\u001b[1;32m      3\u001b[0m       \u001b[0mpipeline_name\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mPIPELINE_NAME\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m      4\u001b[0m       \u001b[0mpipeline_root\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mPIPELINE_ROOT\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m      5\u001b[0m       \u001b[0mdata_root\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mDATA_ROOT\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/portable/tfx_runner.py\u001b[0m in \u001b[0;36mrun\u001b[0;34m(self, pipeline, run_options, **kwargs)\u001b[0m\n\u001b[1;32m    122\u001b[0m     \u001b[0;32melse\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    123\u001b[0m       \u001b[0mrun_options_pb\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 124\u001b[0;31m     \u001b[0;32mreturn\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mrun_with_ir\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mpipeline_pb\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mrun_options\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mrun_options_pb\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/local/local_dag_runner.py\u001b[0m in \u001b[0;36mrun_with_ir\u001b[0;34m(self, pipeline, run_options)\u001b[0m\n\u001b[1;32m    107\u001b[0m           \u001b[0;32mwith\u001b[0m \u001b[0mmetadata\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mMetadata\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mconnection_config\u001b[0m\u001b[0;34m)\u001b[0m \u001b[0;32mas\u001b[0m \u001b[0mmlmd_handle\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    108\u001b[0m             \u001b[0mpartial_run_utils\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0msnapshot\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mmlmd_handle\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mpipeline\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 109\u001b[0;31m         \u001b[0mcomponent_launcher\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mlaunch\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    110\u001b[0m         \u001b[0mlogging\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0minfo\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m'Component %s is finished.'\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mnode_id\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/portable/launcher.py\u001b[0m in \u001b[0;36mlaunch\u001b[0;34m(self)\u001b[0m\n\u001b[1;32m    571\u001b[0m               executor_watcher.address)\n\u001b[1;32m    572\u001b[0m           \u001b[0mexecutor_watcher\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mstart\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 573\u001b[0;31m         \u001b[0mexecutor_output\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_run_executor\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mexecution_info\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    574\u001b[0m       \u001b[0;32mexcept\u001b[0m \u001b[0mException\u001b[0m \u001b[0;32mas\u001b[0m \u001b[0me\u001b[0m\u001b[0;34m:\u001b[0m  \u001b[0;31m# pylint: disable=broad-except\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    575\u001b[0m         execution_output = (\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/portable/launcher.py\u001b[0m in \u001b[0;36m_run_executor\u001b[0;34m(self, execution_info)\u001b[0m\n\u001b[1;32m    446\u001b[0m     \u001b[0moutputs_utils\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mmake_output_dirs\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mexecution_info\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0moutput_dict\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    447\u001b[0m     \u001b[0;32mtry\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 448\u001b[0;31m       \u001b[0mexecutor_output\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_executor_operator\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mrun_executor\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mexecution_info\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    449\u001b[0m       \u001b[0mcode\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mexecutor_output\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mexecution_result\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcode\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    450\u001b[0m       \u001b[0;32mif\u001b[0m \u001b[0mcode\u001b[0m \u001b[0;34m!=\u001b[0m \u001b[0;36m0\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/portable/beam_executor_operator.py\u001b[0m in \u001b[0;36mrun_executor\u001b[0;34m(self, execution_info, make_beam_pipeline_fn)\u001b[0m\n\u001b[1;32m    110\u001b[0m         make_beam_pipeline_fn=make_beam_pipeline_fn)\n\u001b[1;32m    111\u001b[0m     \u001b[0mexecutor\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_executor_cls\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mcontext\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mcontext\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 112\u001b[0;31m     \u001b[0;32mreturn\u001b[0m \u001b[0mpython_executor_operator\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mrun_with_executor\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mexecution_info\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mexecutor\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/portable/python_executor_operator.py\u001b[0m in \u001b[0;36mrun_with_executor\u001b[0;34m(execution_info, executor)\u001b[0m\n\u001b[1;32m     56\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     57\u001b[0m   \u001b[0moutput_dict\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mcopy\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdeepcopy\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mexecution_info\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0moutput_dict\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 58\u001b[0;31m   result = executor.Do(execution_info.input_dict, output_dict,\n\u001b[0m\u001b[1;32m     59\u001b[0m                        execution_info.exec_properties)\n\u001b[1;32m     60\u001b[0m   \u001b[0;32mif\u001b[0m \u001b[0;32mnot\u001b[0m \u001b[0mresult\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/components/statistics_gen/executor.py\u001b[0m in \u001b[0;36mDo\u001b[0;34m(self, input_dict, output_dict, exec_properties)\u001b[0m\n\u001b[1;32m    154\u001b[0m           (split, tfxio_factory(io_utils.all_files_pattern(uri))))\n\u001b[1;32m    155\u001b[0m     \u001b[0;32mif\u001b[0m \u001b[0;32mnot\u001b[0m \u001b[0msplit_and_tfxio\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 156\u001b[0;31m       \u001b[0;32mraise\u001b[0m \u001b[0mValueError\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m'No splits for examples artifact: %s'\u001b[0m \u001b[0;34m%\u001b[0m \u001b[0mexamples\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    157\u001b[0m     \u001b[0;32mwith\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_make_beam_pipeline\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m \u001b[0;32mas\u001b[0m \u001b[0mp\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    158\u001b[0m       \u001b[0;32mfor\u001b[0m \u001b[0msplit\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mtfxio\u001b[0m \u001b[0;32min\u001b[0m \u001b[0msplit_and_tfxio\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;31mValueError\u001b[0m: No splits for examples artifact: Artifact(artifact: id: 18\ntype_id: 14\nuri: \"../artifacts/CopyExampleGen/examples/2/1/\"\ncustom_properties {\n  key: \"is_external\"\n  value {\n    int_value: 1\n  }\n}\ncustom_properties {\n  key: \"state\"\n  value {\n    string_value: \"published\"\n  }\n}\ncustom_properties {\n  key: \"tfx_version\"\n  value {\n    string_value: \"1.12.0\"\n  }\n}\nstate: LIVE\ncreate_time_since_epoch: 1676047493360\nlast_update_time_since_epoch: 1676047493360\n, artifact_type: id: 14\nname: \"Examples\"\nproperties {\n  key: \"span\"\n  value: INT\n}\nproperties {\n  key: \"split_names\"\n  value: STRING\n}\nproperties {\n  key: \"version\"\n  value: INT\n}\nbase_type: DATASET\n)"
}

# print(dict_output["stack"])

ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
result = ansi_escape.sub('', dict_output["stack"])

print(result)


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
/tmp/ipykernel_2151988/3420384479.py in <cell line: 1>()
----> 1 tfx.orchestration.LocalDagRunner().run(
      2   _create_pipeline(
      3       pipeline_name=PIPELINE_NAME,
      4       pipeline_root=PIPELINE_ROOT,
      5       data_root=DATA_ROOT,

~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/portable/tfx_runner.py in run(self, pipeline, run_options, **kwargs)
    122     else:
    123       run_options_pb = None
--> 124     return self.run_with_ir(pipeline_pb, run_options=run_options_pb, **kwargs)

~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/local/local_dag_runner.py in run_with_ir(self, pipeline, run_options)
    107           with metadata.Metadata(connection_config) as mlmd_handle:
    108             partial_run_utils.snapshot(mlmd_handle, pipeli

In [ ]:
# [SPLIT_TRAIN] View Dataset Artifact as tf.record
train_uri = os.path.join('./artifacts/CopyExampleGen/examples/1/', 'Split-train')

tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset:
  # Prints out tf.record
  print(tfrecord)
  
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()

  # Prints out parsed tfrecord as JSON
  example.ParseFromString(serialized_example)
  print(example)


In [ ]:
# [SPLIT_EVAL] View Dataset Artifact
eval_uri = os.path.join('./artifacts/CsvExampleGen/examples/1/', 'Split-eval')

tfrecord_filenames = [os.path.join(eval_uri, name)
                      for name in os.listdir(eval_uri)]

print(tfrecord_filenames)


dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(3):
  # Prints out tf.record
  print(tfrecord)

  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  
  # Prints out parsed tfrecord as JSON
  example.ParseFromString(serialized_example)
  print(example)